Youtube Demo
https://youtu.be/VCfWaaida1c

取得公鑰，加入到容器中

In [ ]:
%env SSH_SERVER=
%env SSH_PORT=
!ssh-keygen -t rsa -q -f "$HOME/.ssh/id_rsa" -N ""
!apt-get -y update && apt-get -y install openssh-server

In [ ]:
!cat ~/.ssh/id_rsa.pub

啟動OpenSSH Server
在容器建立金鑰，並將公鑰設定回colab VM。


In [ ]:
!mkdir -p /run/sshd
!/usr/sbin/sshd
!nohup ssh -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa -p ${SSH_PORT} colab@${SSH_SERVER} -NR \*:2222:0.0.0.0:22&
!ssh -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa -p ${SSH_PORT} colab@${SSH_SERVER} "ssh-keygen -t rsa -q -f /home/colab/.ssh/id_rsa -N ''"
!scp -P ${SSH_PORT} colab@${SSH_SERVER}:~/.ssh/id_rsa.pub ~/.ssh/authorized_keys

可以直接用如下指令，在host端連回colab vm
```
docker-compose exec -ucolab colab ssh colab
```



安裝及啟動code-server

In [ ]:
!curl -fsSL https://code-server.dev/install.sh | sh

Code Server不使用SSL

In [ ]:
!nohup code-server --bind-addr 0.0.0.0:8081&
!nohup ssh -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa -p ${SSH_PORT} colab@${SSH_SERVER} -NR \*:8080:0.0.0.0:8081&

Code-Server 使用SSL版本
需要copy憑證fullchain.pem及privkey.pem到/content內。

In [ ]:
!nohup code-server --cert fullchain.pem --cert-key privkey.pem --bind-addr 0.0.0.0:8081&
!nohup ssh -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa -p ${SSH_PORT} colab@${SSH_SERVER} -NR \*:8080:0.0.0.0:8081&

顯示code-server密碼

In [ ]:
!cat /root/.config/code-server/config.yaml

In [ ]:
import easyocr
reader = easyocr.Reader(['ch_tra','en']) # need to run only once to load model into memory
result = reader.readtext('chinese.png')
print(result)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')